---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

def blight_model():
    
    # Your code here
    cols = ['agency_name', 'fine_amount', 'discount_amount', 'judgment_amount', 'violation_code', 'disposition']
    data = pd.read_csv('test.csv')
    data_test = data[cols]
    
    df = pd.read_csv('train.csv', encoding = 'ISO-8859-1', low_memory = False)
    df = df[(df['compliance'] == 0) | (df['compliance'] == 1)]
    X = df[cols]
    y = df['compliance']
    
    le1 = LabelEncoder()
    X.loc[:, 'agency_name'] = le1.fit_transform(X.loc[:, 'agency_name'])
    data_test.loc[:, 'agency_name'] = le1.transform(data_test.loc[:, 'agency_name'])
    
    le2 = LabelEncoder()
    le2.fit(X.loc[:, 'violation_code'].append(data_test.loc[:, 'violation_code'], ignore_index=True))
    X.loc[:, 'violation_code'] = le2.transform(X.loc[:, 'violation_code'])
    data_test.loc[:, 'violation_code'] = le2.transform(data_test.loc[:, 'violation_code'])
    
    le3 = LabelEncoder()
    le3.fit(X.loc[:, 'disposition'].append(data_test.loc[:, 'disposition'], ignore_index=True))
    X.loc[:, 'disposition'] = le3.transform(X.loc[:, 'disposition'])
    data_test.loc[:, 'disposition'] = le3.transform(data_test.loc[:, 'disposition'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, random_state=0)
    y_scores = clf.fit(X_train, y_train).predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_scores[:, 1])
    auroc = auc(fpr, tpr)
    
    pred = clf.predict_proba(data_test)
    ans = pd.Series(data = pred[:, 1], index = data['ticket_id'])
    
    return ans # Your answer here

In [2]:
blight_model()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ticket_id
284932    0.136165
285362    0.014137
285361    0.053870
285338    0.047390
285346    0.081470
285345    0.047390
285347    0.055486
285342    0.769906
285530    0.014137
284989    0.021619
285344    0.052133
285343    0.014137
285340    0.014137
285341    0.052133
285349    0.081470
285348    0.047390
284991    0.021619
285532    0.021619
285406    0.021619
285001    0.019784
285006    0.016197
285405    0.014137
285337    0.021619
285496    0.052133
285497    0.047390
285378    0.014137
285589    0.021619
285585    0.047390
285501    0.053870
285581    0.014137
            ...   
376367    0.030642
376366    0.035155
376362    0.188130
376363    0.216556
376365    0.030642
376364    0.035155
376228    0.035155
376265    0.035155
376286    0.271540
376320    0.035155
376314    0.035155
376327    0.271540
376385    0.271540
376435    0.100664
376370    0.878160
376434    0.077427
376459    0.070080
376478    0.010143
376473    0.035155
376484    0.015967
376482    0.018259
37

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

def another_example_blight_model():
    
    # Your code here
    cols = ['fine_amount', 'discount_amount', 'judgment_amount']
    data = pd.read_csv('test.csv')
    data_test = data[cols]
    
    df = pd.read_csv('train.csv', encoding = 'ISO-8859-1', low_memory = False)
    df = df[(df['compliance'] == 0) | (df['compliance'] == 1)]
    X = df[cols]
    y = df['compliance']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    
    clf = LogisticRegression(C=100)
    y_scores = clf.fit(X_train, y_train).predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_scores[:, -1])
    auroc = auc(fpr, tpr)
    
    pred = clf.predict_proba(data_test)
    ans = pd.Series(data = pred[:, -1], index = data['ticket_id'])
    
    return ans #auroc # Your answer here
another_example_blight_model()

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


ticket_id
284932    5.742813e-02
285362    7.439028e-03
285361    7.336287e-02
285338    5.742813e-02
285346    7.336287e-02
285345    5.742813e-02
285347    8.277876e-02
285342    1.952016e-01
285530    7.439028e-03
284989    2.701755e-02
285344    8.277876e-02
285343    7.439028e-03
285340    7.439028e-03
285341    8.277876e-02
285349    7.336287e-02
285348    5.742813e-02
284991    2.701755e-02
285532    2.701755e-02
285406    2.701755e-02
285001    2.701755e-02
285006    7.439028e-03
285405    7.439028e-03
285337    2.701755e-02
285496    8.277876e-02
285497    5.742813e-02
285378    7.439028e-03
285589    2.701755e-02
285585    5.742813e-02
285501    7.336287e-02
285581    7.439028e-03
              ...     
376367    1.422098e-02
376366    5.073631e-02
376362    5.073631e-02
376363    5.742813e-02
376365    1.422098e-02
376364    5.073631e-02
376228    5.073631e-02
376265    5.073631e-02
376286    2.310937e-01
376320    5.073631e-02
376314    5.073631e-02
376327    2.310937e-01
3